# ETL (Extract, Load, Transform)

En el doloroso escenario de la realidad argentina, donde anualmente se despiden cerca de 4.000 vidas en siniestros viales, nuestro viaje comienza con una misión fundamental: transformar datos en entendimiento, desvelar los mensajes implícitos y, sobre todo, contribuir a la reducción de tragedias en sus carreteras.

En éste proceso de ETL nos embarcaremos en la tarea de explorar dos archivos en formato Excel, testigos silenciosos de eventos trágicos: **'homicidios.xlsx'** y **'lesiones.xlsx'**. Cada hoja de trabajo, ya sea en homicidios (HECHOS y VICTIMAS) o lesiones (HECHOS y VICTIMAS), guarda consigo relatos que claman por ser desentrañados.

Este notebook no es meramente código; es una odisea de descubrimiento, un esfuerzo meticuloso para revelar los secretos que se ocultan en los datos crudos. Nos sumergiremos en cada fila, exploraremos cada columna, y extraeremos la esencia que yace en cada celda. La transformación de información cruda en conocimiento es la brújula que nos guía hacia la comprensión y, más importante aún, hacia la acción para mitigar las pérdidas humanas en las carreteras de la Ciudad de Buenos Aires.

## 1. Importar Librerías

In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from functions import cargar_datos_desde_excel
from functions import data_cleaning
from functions import analizar_valores_sd

## 2. Cargar Datos

In [68]:
# Invoca la función 'cargar_datos_desde_excel'

datos_homicidios = cargar_datos_desde_excel('homicidios.xlsx', ['HECHOS', 'VICTIMAS'])
datos_lesiones = cargar_datos_desde_excel('lesiones.xlsx', ['HECHOS', 'VICTIMAS'])

## 3. Explorar y limpiar datos

### 3.1. Dataset: Homicidios - Hechos

In [69]:
# Acceder al DataFrame por nombre de hoja, Homicidios - Hechos

h_hechos_df = datos_homicidios['HECHOS']

In [70]:
# Visualiza las primeras filas del Dataframe

print("Original DataFrame:")
h_hechos_df.head(3)

Original DataFrame:


,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [71]:
# Obtener información general del DataFrame

h_hechos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Tras verificar el diccionario de datos, identificamos que los valores "SD" corresponden a "Sin datos". Por consiguiente, tomamos la decisión de cambiar estos valores a NaN para estandarizar.

In [72]:
# Invoca la función 'analizar_valores_sd'

resultados_h_hechos = analizar_valores_sd(h_hechos_df)
resultados_h_hechos

,Columna,Cantidad de SD,Porcentaje de SD
6,HORA,1,0.143678
7,HH,1,0.143678
8,LUGAR_DEL_HECHO,1,0.143678
19,VICTIMA,9,1.293103
20,ACUSADO,23,3.304598


In [73]:
# Reemplazar 'SD' por NaN en todo el DataFrame

h_hechos_df.replace(['SD','sd'], np.nan, inplace=True)

In [74]:
# Iniciamos la preparación de los diccionarios y listas que alimentaran la función 'data_cleaning' 

In [75]:
# Columnas para pasar a minúcula

columns_to_lower = ['TIPO_DE_CALLE', 'VICTIMA', 'ACUSADO']

Convertir todos los datos a minúsculas evita posibles problemas de coincidencia y simplificas las operaciones de búsqueda y filtrado.

In [76]:
# Columnas para eliminar

columns_to_drop = ['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO','Calle','Altura',
                   'Cruce','Dirección Normalizada','XY (CABA)','PARTICIPANTES']

Decidimos eliminar las columnas 'AAAA', 'MM', y 'DD' del conjunto de datos, ya que la información de fecha ya está presente en la columna 'FECHA'. Esta decisión simplifica el conjunto de datos, mantiene una estructura más consistente y estandarizada, ahorra espacio y facilita el análisis temporal.

Asimismo, eliminamos la columna 'HORA' debido a la redundancia de la información de hora, que ya está contenida en la columna 'HH'. Esta elección nos proporciona una visión más general y simplificada de la distribución de los siniestros a lo largo del día. 

Las columnas 'LUGAR_DEL_HECHO','Calle','Altura','Cruce' y 'Dirección Normalizada' implicaban datos de la dirección del hecho incluso, 'XY (CABA)' que contenía coordenadas en formato de proyección cartesiana también fue eliminada. Preferimos trabajar con las columnas de **latitud y longitud** por su mayor intuición, no tiene datos nulos, familiaridad para la mayoría de las personas, facilidad de representación gráfica en mapas y conformidad con estándares de sistemas de información geográfica (SIG).

La columna 'PARTICIPANTES' contenia información concatenada redundante de 'VICTIMA' y 'ACUSADO', así que se mantuvieron éstas dos últimas.

In [77]:
# Renombrar columnas

rename_dict = {'ID': 'id_siniestro', 'N_VICTIMAS': 'nro_victimas', 'FECHA': 'fecha', 'HH': 'franja_hora',
               'TIPO_DE_CALLE': 'tipo_calle', 'COMUNA': 'comuna', 'pos x': 'longitud', 'pos y': 'latitud',
               'VICTIMA': 'vehiculo_victima', 'ACUSADO': 'vehiculo_acusado'}

Renombramos columnas para mejorar la claridad y legibilidad del conjunto de datos.

In [78]:
# Convertir a entero

columns_to_int = [ 'nro_victimas', 'franja_hora', 'comuna']

In [79]:
# Categorización de columnas

columns_to_categorize = ['franja_hora', 'tipo_calle', 'comuna', 'vehiculo_victima', 'vehiculo_acusado']

Al convertir columnas con un conjunto limitado de valores a categorías, se puede facilitar el análisis exploratorio de datos y la comprensión de las distribuciones de esos valores.

In [80]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

h_hechos_dfcleaned = data_cleaning(h_hechos_df,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    categorize_columns=columns_to_categorize # Categorizar columnas
                                   )

En el conjunto de datos tenemos dos registros con comuna valor 0. Para este registro '2016-0174' se tenía la calle y, con ésta logramos obtener comuna, longitud y latitud . Para el caso del registro '2016-0151' no tuvimos la misma suerte, por lo cual buscamos el siniestro más cercano '2016-0155' que haya ocurrido también en una calle y le imputamos el mismo valor a las columnas comuna, longitud y latitud. Por la mínima cantidad de registros no vimos necesario hacer uso de alguna api, obtuvimos los datos consultando esta web https://www.comollegara.com/direccion-a-lat-long.html?lugar=autopista%2025%20de%20mayo%20buenos%20aires.

In [81]:
# Reemplazar valores

# Definir los registros que deseas reemplazar
registros_a_reemplazar = [
    {'id_siniestro': '2016-0174', 'comuna': 7, 'longitud': -58.369529920, 'latitud': -34.622472560}, # Comunas 0 
    {'id_siniestro': '2016-0151', 'comuna': 10, 'longitud': -58.52756365, 'latitud': -34.62772274}, # Comunas 0 
    {'id_siniestro': '2018-0039', 'comuna': 14, 'longitud': -58.457579360, 'latitud': -34.538957930}, #sin longitud/latitud
    {'id_siniestro': '2020-0026', 'comuna': 14, 'longitud': -58.457579360, 'latitud': -34.538957930}, #sin longitud/latitud  
    {'id_siniestro': '2017-0042', 'comuna': 14, 'longitud': -58.457579360, 'latitud': -34.538957930}, #sin longitud/latitud  
    {'id_siniestro': '2017-0050', 'comuna': 9, 'longitud': -58.481800300, 'latitud': -34.647005340}, #sin longitud/latitud
    {'id_siniestro': '2017-0051', 'comuna': 7, 'longitud': -58.479447760, 'latitud': -34.678121010}, #sin longitud/latitud      
    {'id_siniestro': '2017-0140', 'comuna': 4, 'longitud': -58.380781350, 'latitud': -34.622364700}, #sin longitud/latitud
    {'id_siniestro': '2020-0039', 'comuna': 9, 'longitud': -58.483096200, 'latitud': -34.671405150}, #sin longitud/latitud   
    {'id_siniestro': '2021-0023', 'comuna': 4, 'longitud': -58.364905600, 'latitud': -34.622974220}, #sin longitud/latitud  
    {'id_siniestro': '2016-0052', 'comuna': 13, 'longitud': -58.457579360, 'latitud': -34.538957930}, #sin longitud/latitud   
    {'id_siniestro': '2016-0136', 'comuna': 4, 'longitud': -58.363401550, 'latitud': -34.624371200}, #sin longitud/latitud  
] 

# Iterar sobre los registros y realizar el reemplazo
for registro in registros_a_reemplazar:
    id_siniestro = registro['id_siniestro']
    comuna = registro['comuna']
    longitud = registro['longitud']
    latitud = registro['latitud']

    # Utilizar loc para reemplazar los valores específicos
    h_hechos_dfcleaned.loc[h_hechos_dfcleaned['id_siniestro'] == id_siniestro, 'comuna'] = comuna
    h_hechos_dfcleaned.loc[h_hechos_dfcleaned['id_siniestro'] == id_siniestro, 'longitud'] = longitud
    h_hechos_dfcleaned.loc[h_hechos_dfcleaned['id_siniestro'] == id_siniestro, 'latitud'] = latitud

In [82]:
print("\nCleaned DataFrame:")
h_hechos_dfcleaned.head()


Cleaned DataFrame:


,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado
0,2016-0001,1,2016-01-01,4,avenida,8,-58.47533969,-34.68757022,moto,auto
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros
2,2016-0003,1,2016-01-03,7,avenida,1,-58.39040293,-34.63189362,moto,auto
3,2016-0004,1,2016-01-10,0,avenida,8,-58.46503904,-34.68092974,moto,NaN
4,2016-0005,1,2016-01-21,5,avenida,1,-58.38718297,-34.62246630,moto,pasajeros


--------------------------------------------------------------------------------------------------------------------------------

### 3.2. Dataset: Homicidios - Víctimas

In [83]:
# Acceder al DataFrame por nombre de hoja, Homicidios - Víctimas

h_victimas_df = datos_homicidios['VICTIMAS']

In [84]:
# Visualiza las primeras filas del Dataframe

print("Original DataFrame:")
h_victimas_df.head(3)

Original DataFrame:


,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00


In [85]:
# Obtener información general del DataFrame

h_victimas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


Tras verificar el diccionario de datos, identificamos que los valores "SD" corresponden a "Sin datos". Por consiguiente, tomamos la decisión de cambiar estos valores a NaN para estandarizar.

In [86]:
# Invoca la función 'analizar_valores_sd'

resultados_h_victimas = analizar_valores_sd(h_victimas_df)
resultados_h_victimas

,Columna,Cantidad de SD,Porcentaje de SD
5,ROL,11,1.534170
6,VICTIMA,9,1.255230
7,SEXO,6,0.836820
8,EDAD,53,7.391911
9,FECHA_FALLECIMIENTO,68,9.483961


In [87]:
# Reemplazar 'SD' por NaN en todo el DataFrame

h_victimas_df.replace(['SD','sd'], np.nan, inplace=True)

In [88]:
# Iniciamos la preparación de los diccionarios y listas que alimentaran la función 'data_cleaning' 

In [89]:
# Columnas para pasar a minúcula

columns_to_lower = ['SEXO','ROL']

Convertir todos los datos a minúsculas evita posibles problemas de coincidencia y simplificas las operaciones de búsqueda y filtrado.

In [90]:
# Columnas para eliminar

columns_to_drop = ['FECHA','AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO', 'VICTIMA']

Las columnas que decidimos eliminar nos parecen irrelevantes para nuestro análisis. Algunas de ellas se complementan con los datos de la hoja 'HECHO'

In [91]:
# Renombrar columnas

rename_dict = {'ID_hecho': 'id_siniestro', 'SEXO': 'sexo',
               'EDAD': 'edad', 'ROL': 'rol'}

Renombramos columnas para mejorar la claridad y legibilidad del conjunto de datos.

In [92]:
# Convertir a entero

columns_to_int = ['edad']

In [93]:
# Columnas para ser agregadas

new_columns_dict = {'gravedad': 'fatal'}

In [94]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

h_victimas_dfcleaned = data_cleaning(h_victimas_df,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas 
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    new_columns=new_columns_dict, # Agregar columnas
                                    )

In [95]:
# Categorización de columnas

columns_to_categorize = ['sexo','edad','gravedad','rol']
h_victimas_dfcleaned = data_cleaning(h_victimas_dfcleaned, categorize_columns=columns_to_categorize) 

Al convertir columnas con un conjunto limitado de valores a categorías, se puede facilitar el análisis exploratorio de datos y la comprensión de las distribuciones de esos valores.

In [96]:
print("\nCleaned DataFrame:")
h_victimas_dfcleaned.head()


Cleaned DataFrame:


,id_siniestro,rol,sexo,edad,gravedad
0,2016-0001,conductor,masculino,19,fatal
1,2016-0002,conductor,masculino,70,fatal
2,2016-0003,conductor,masculino,30,fatal
3,2016-0004,conductor,masculino,18,fatal
4,2016-0005,conductor,masculino,29,fatal


--------------------------------------------------------------------------------------------------------------------------------

### 3.3. Dataset: Lesiones - Hechos

In [97]:
# Acceder al DataFrame por nombre de hoja, Lesiones - Hechos

l_hechos_df = datos_lesiones['HECHOS']

In [98]:
# Visualiza las primeras filas del Dataframe

print("Original DataFrame:")
l_hechos_df.head(3)

Original DataFrame:


,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,-34.559658,CICLISTA,SD,CICLISTA-SD,SD,SD,SD,SD,x,SD
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,-34.669125,AUTO,SD,AUTO-SD,SD,x,SD,SD,SD,SD
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,-34.677556,SD,SD,SD-SD,SD,SD,SD,SD,SD,SD


In [99]:
# Obtener información general del DataFrame

l_hechos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

Tras verificar el diccionario de datos, identificamos que los valores "SD" corresponden a "Sin datos". Por consiguiente, tomamos la decisión de cambiar estos valores a NaN para estandarizar.

In [100]:
# Invoca la función 'analizar_valores_sd'

resultados_l_hechos = analizar_valores_sd(l_hechos_df)
resultados_l_hechos

,Columna,Cantidad de SD,Porcentaje de SD
6,hora,4,0.016817
8,direccion_normalizada,10815,45.469834
9,comuna,846,3.556864
10,tipo_calle,11045,46.436830
11,otra_direccion,18295,76.918226
15,geocodificacion_CABA,1213,5.099853
16,longitud,1209,5.083036
17,latutid,1209,5.083036
18,victima,10733,45.125079
19,acusado,15288,64.275804


In [101]:
# Reemplazar 'SD' por NaN en todo el DataFrame

l_hechos_df.replace(['SD','sd'], np.nan, inplace=True)

In [102]:
# Iniciamos la preparación de los diccionarios y listas que alimentaran la función 'data_cleaning' 

In [103]:
# Columnas para pasar a minúcula

columns_to_lower = ['tipo_calle', 'victima', 'acusado', 'gravedad']

Convertir todos los datos a minúsculas evita posibles problemas de coincidencia y simplificas las operaciones de búsqueda y filtrado.

In [104]:
# Columnas para eliminar

columns_to_drop = ['aaaa','mm', 'dd', 'hora', 'direccion_normalizada', 'otra_direccion', 'calle', 'altura', 'cruce',
                   'participantes', 'geocodificacion_CABA', 'moto', 'auto', 'transporte_publico', 'camion', 'ciclista']

In [105]:
# Reemplazar valores nulos por otros valores

fill_na_dict = {'gravedad': 'leve'}

Sobre la gravedad de las lesiones provocadas, según diccionario de datos a efectos analíticos, los casos sin datos se corresponden con una alta probabilidad a casos leves.

In [106]:
# Renombrar columnas

rename_dict = {'id': 'id_siniestro', 'n_victimas': 'nro_victimas',
               'latutid': 'latitud', 'victima': 'vehiculo_victima', 'acusado': 'vehiculo_acusado'}

Renombramos columnas para mejorar la claridad y legibilidad del conjunto de datos.

In [107]:
# Confirmo la existencia de valor igual a 0 en nro_victimas

registros_nro_victimas_0 = l_hechos_df[l_hechos_df['n_victimas'] == 0]
registros_nro_victimas_0

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
9928,PNA-2019-0005246,0,2019,12,20,2019-12-20 00:00:00,NaN,NaN,27 DE FEBRERO AV. 6300,NaN,...,NaN,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN


Para lo que corresponde al hecho, no hay mayores detalles en el registro resultante con victimas 0 casi todas sus columnas se encuentran con valores NaN ; sin embargo, al consultar las victimas en el archivo lesiones.xlsx (hoja victima que será tratada más adelante) se evidencia que se trató de una mujer de 33 años por lo cual procedo a reemplazar el valor por 1.

In [108]:
# Reemplazar valores en columnas

values_to_replace = {'comuna': {'No Especificada': np.nan},
                     'nro_victimas': {0: 1}
                    }

In [109]:
# Convertir a flotante

columns_to_float = [ 'latitud', 'longitud']

In [110]:
# Convertir a entero

columns_to_int = [ 'nro_victimas', 'franja_hora', 'comuna']

In [111]:
# Categorización de columnas

columns_to_categorize = ['franja_hora', 'tipo_calle', 'comuna', 'vehiculo_victima', 'vehiculo_acusado', 'gravedad']

Al convertir columnas con un conjunto limitado de valores a categorías, se puede facilitar el análisis exploratorio de datos y la comprensión de las distribuciones de esos valores.

In [112]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

l_hechos_dfcleaned = data_cleaning(l_hechos_df,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    fill_na=fill_na_dict, # Rellenar nulos
                                    rename_columns=rename_dict,  # Renombrar columnas                            
                                    replace_values=values_to_replace, # Reemplazar valores                                   
                                    convert_to_float=columns_to_float, # Convertir a float
                                    convert_to_int_columns=columns_to_int, #Conversión entero
                                    categorize_columns=columns_to_categorize # Categorizar columnas
                                   )

In [113]:
print("\nCleaned DataFrame:")
l_hechos_dfcleaned.head()


Cleaned DataFrame:


,id_siniestro,nro_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latitud,vehiculo_victima,vehiculo_acusado,gravedad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9,14,NaN,-58.408911,-34.559658,ciclista,NaN,leve
1,LC-2019-0000053,1,2019-01-01 00:00:00,1,8,NaN,-58.443510,-34.669125,auto,NaN,leve
2,LC-2019-0000063,1,2019-01-01 00:00:00,2,8,NaN,-58.468335,-34.677556,NaN,NaN,leve
3,LC-2019-0000079,1,2019-01-01 00:00:00,2,7,NaN,-58.437425,-34.647349,peaton,NaN,leve
4,LC-2019-0000082,4,2019-01-01 00:00:00,4,3,NaN,-58.398225,-34.604579,auto,NaN,leve


--------------------------------------------------------------------------------------------------------------------------------

## 3.4. Dataset: Lesiones - Víctimas

In [114]:
# Acceder al DataFrame por nombre de hoja, Lesiones - Víctimas

l_victimas_df = datos_lesiones['VICTIMAS']

In [115]:
# Visualiza las primeras filas del Dataframe

print("Original DataFrame:")
l_victimas_df.head(3)

Original DataFrame:


,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,SD,SD
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,SD,SD


In [116]:
# Obtener información general del DataFrame

l_victimas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


In [117]:
# Invoca la función 'analizar_valores_sd'

resultados_l_victimas = analizar_valores_sd(l_victimas_df)
resultados_l_victimas

,Columna,Cantidad de SD,Porcentaje de SD
5,VEHICULO_VICTIMA,4479,16.225321
6,SEXO,1929,6.987865
7,EDAD_VICTIMA,3990,14.453903
8,GRAVEDAD,20722,75.066111


In [118]:
# Reemplazar 'SD' por NaN en todo el DataFrame

l_victimas_df.replace(['SD','sd'], np.nan, inplace=True)

Tras verificar el diccionario de datos, identificamos que los valores "SD" corresponden a "Sin datos". Por consiguiente, tomamos la decisión de cambiar estos valores a NaN para estandarizar.

In [119]:
# Iniciamos la preparación de los diccionarios y listas que alimentaran la función 'data_cleaning' 

In [120]:
# Columnas para pasar a minúcula

columns_to_lower = ['SEXO']

Convertir todos los datos a minúsculas evita posibles problemas de coincidencia y simplificas las operaciones de búsqueda y filtrado.

In [121]:
# Columnas para eliminar

columns_to_drop = ['FECHA ','AAA', 'MM', 'DD','VEHICULO_VICTIMA','GRAVEDAD']

Las columnas que decidimos eliminar nos parecen irrelevantes para nuestro análisis. Algunas de ellas se complementan con los datos de la hoja 'HECHO'


In [122]:
# Renombrar columnas

rename_dict = {'ID hecho': 'id_siniestro', 'SEXO': 'sexo', 'EDAD_VICTIMA': 'edad'}

Renombramos columnas para mejorar la claridad y legibilidad del conjunto de datos.

In [123]:
# Reemplazar valores en columnas

values_to_replace = {'sexo': {'varon': 'masculino', 'mujer': 'femenino'}}

In [124]:
# Categorización de columnas

columns_to_categorize = ['sexo','edad']

Al convertir columnas con un conjunto limitado de valores a categorías, se puede facilitar el análisis exploratorio de datos y la comprensión de las distribuciones de esos valores.

In [125]:
# Columnas para ser agregadas

new_columns_dict = {'rol': ''}

In [126]:
# Invocamos la función 'data_cleaning' para que haga el proceso de limpieza de los datos 

l_victimas_dfcleaned = data_cleaning(l_victimas_df,
                                    strip_spaces=True, # Eliminar espacios en blanco
                                    lowercase_columns=columns_to_lower, # Convertir a minúsculas                                    
                                    drop_columns=columns_to_drop,  # Eliminar columnas
                                    rename_columns=rename_dict,  # Renombrar columnas 
                                    replace_values=values_to_replace, # Reemplazar valores
                                    categorize_columns=columns_to_categorize, # Categorizar columnas
                                    new_columns=new_columns_dict, # Agregar columnas
                                    )

In [127]:
print("\nCleaned DataFrame:")
l_victimas_dfcleaned.head()


Cleaned DataFrame:


,id_siniestro,sexo,edad,rol
0,LC-2019-0000053,masculino,57,
1,LC-2019-0000063,NaN,NaN,
2,LC-2019-0000079,masculino,NaN,
3,LC-2019-0000082,masculino,45,
4,LC-2019-0000082,femenino,45,


## 4. Combinar Datos

In [128]:
# Fusionar DataFrames

merged_df1 = pd.merge(h_hechos_dfcleaned, h_victimas_dfcleaned, left_on='id_siniestro', right_on='id_siniestro', how='inner')
merged_df2 = pd.merge(l_hechos_dfcleaned, l_victimas_dfcleaned, left_on='id_siniestro', right_on='id_siniestro', how='inner')

In [129]:
# Reorganizar columnas merged_df2

reorden = ['id_siniestro', 'nro_victimas', 'fecha', 'franja_hora', 'tipo_calle', 'comuna', 'longitud', 'latitud', 
           'vehiculo_victima', 'vehiculo_acusado', 'rol', 'sexo', 'edad', 'gravedad']

merged_df2 = merged_df2[reorden]

In [130]:
merged_df1.head(3)

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado,rol,sexo,edad,gravedad
0,2016-0001,1,2016-01-01,4,avenida,8,-58.47533969,-34.68757022,moto,auto,conductor,masculino,19,fatal
1,2016-0002,1,2016-01-02,1,gral paz,9,-58.50877521,-34.66977709,auto,pasajeros,conductor,masculino,70,fatal
2,2016-0003,1,2016-01-03,7,avenida,1,-58.39040293,-34.63189362,moto,auto,conductor,masculino,30,fatal


In [131]:
merged_df2.head(3)

,id_siniestro,nro_victimas,fecha,franja_hora,tipo_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado,rol,sexo,edad,gravedad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9,NaN,14,-58.408911,-34.559658,ciclista,NaN,,NaN,NaN,leve
1,LC-2019-0000053,1,2019-01-01 00:00:00,1,NaN,8,-58.443510,-34.669125,auto,NaN,,masculino,57,leve
2,LC-2019-0000063,1,2019-01-01 00:00:00,2,NaN,8,-58.468335,-34.677556,NaN,NaN,,NaN,NaN,leve


## 6. Almacenamiento de Datos

In [132]:
# Almacenar los DataFrames finales en un nuevo archivo CSV

merged_df1.to_csv('homicidios_cleaned.csv', index=False)
merged_df2.to_csv('lesiones_cleaned.csv', index=False)